### Introdução ao uso de LangChain

O Langchain permite trabalhar facilmente com diversos modelos.

**Modelos:**

- [Documentação LangChain](https://python.langchain.com/docs/integrations/llms/)

### Instalação

In [1]:
import torch
import os
import getpass # Para informar o token do huggingFace

# Transformers do HuggingFace
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, AwqConfig

# Integrando Langchain juntamente com o Hugging Face
from langchain_huggingface import HuggingFacePipeline

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate
)

from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv

/home/wolf/anaconda3/envs/ai_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-21 23:31:39.238491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761100299.261633   21584 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761100299.268628   21584 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761100299.286685   21584 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than o

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

### Carregando LLM via pipeline (a mesma da aula anterior)

**Utilizando Langchain juntamente com o HuggingFace**

Aqui iremos utilizar o mesmo código e modelo da aula anterior, porém, utilizando o Langchain para pré processar os textos, sem a necessidade de criar templates como fizemos nas aulas com HuggingFace.

In [10]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, # Modelo que será baixado
    device_map = "cuda", # Deve ser carregada em uma GPU habilitada para CUDA
    torch_dtype = "auto",
    trust_remote_code = False,
    attn_implementation="eager" # Método de implementação para o mecanismo de atenção
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [02:01<00:00, 60.75s/it]


A temperatura controla a aleatoriedade do processo de geração de texto.

Valores mais baixos tornam a saída do modelo mais determinística, enquanto que valores mais altos aumentam a "criatividade" do algoritimo (algo que também pode causar a alucinação).

In [12]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500, # Será gerado no máximo 500 tokens
    do_sample = True, # Adiciona fator de aleatoriedade na geração dos textos
    repetition_penalty = 1.1, # Desencoraja o modelo a gerar textos ou frases muito repetitivas. 1 é padrão (nenhuma penalidade)
    return_full_text = False # Determina se deve ser retornado o texto completo, incluindo o prompt completo ou apenas o texto gerado
)

Device set to use cuda


In [13]:
llm = HuggingFacePipeline(pipeline=pipe)

In [14]:
output = llm.invoke(input="Qual é o sentido da vida?")
print(output)



### Response:O significado ou propósito da vida tem sido objeto de debate filosófico, espiritual e científico por séculos. A resposta a essa pergunta pode variar muito dependendo das crenças pessoais, culturais e religiosas individuais. Algumas pessoas encontram significado em suas conexões com outras pessoas, contribuições para sua comunidade ou realização de seus próprios sonhos e metas. Outras buscam significado através do crescimento pessoal, autoaperfeiçoamento ou experiências espirituais profundas. Em última análise, muitos argumentam que cada indivíduo deve criar seu próprio senso de significado na vida, explorando as paixões, valores e interesses únicos deles mesmos.


### Outros modelos Open Source (Llama)

- [Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)

Com a máquina utilizada para fazer o curso, não foi possível baixar e utilizar o modelo llama a partir do HuggingFace nem com a versão quantizada, iremos continuar utilizando o modelo *Phi-3*.

In [ ]:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"

### Modelos de Chat

In [9]:
from langchain_core.messages import (HumanMessage, SystemMessage)
from langchain_huggingface import ChatHuggingFace

#### **Adequando um modelo**

Aqui estamos definindo como o modelo irá se comportar. Diferentemente dos exemplos onde precisavamos definir as tags de finalização dos textos e quem era o ator (role) manualmente, desta vez podemos definir com as classes *SystemMessage* e *HumanMessage*.

In [16]:
msgs = [
    SystemMessage(content="Você é um assistente e está respondendo perguntas gerais."),
    HumanMessage(content="Explique para mim brevemente o que seria a vida.")
]

In [17]:
chat_model = ChatHuggingFace(llm=llm) # Aqui é definido qual LLM será utilizada, no momento, estamos utilizando o Phi-3

Abaixo é o template do modelo.

In [19]:
model_template = tokenizer.chat_template
model_template

"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

Abaixo é o template preenchido com os prompts.

In [20]:
chat_model._to_chat_prompt(msgs)

'<|system|>\nVocê é um assistente e está respondendo perguntas gerais.<|end|>\n<|user|>\nExplique para mim brevemente o que seria a vida.<|end|>\n<|assistant|>\n'

In [23]:
resp = chat_model.invoke(msgs)
print(resp.content)

 A vida, em seu sentido mais amplo, refere-se à existência de seres vivos - organismos capazes de crescer, se reproduzir, responder ao ambiente e manter uma ordem interna por meio do metabolismo. Ela abrange desde as formas simples como bactérias até os complexos sistemas biológicos encontrados nas plantas e animais. Cada forma de vida possui características únicas adaptadas aos seus habitats específicos, permitindo sua sobrevivência e evolução ao longo do tempo geológico. Em essência, a vida representa toda a diversidade da natureza viva presente no planeta Terra.
